In [1]:
import copy
import math
import struct
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import math
import csv
from scipy.interpolate import interp1d
from scipy.stats import pearsonr, mode
from scipy.signal import savgol_filter
import xgboost as xgb
import sklearn
import copy
import warnings

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from scipy.ndimage import gaussian_filter1d
import struct

addressPrefix='C:/Users/sorush.omidvar/Google Drive/Documents/Educational/TAMU/Research/CGM Dataset/Hoover/'
if not os.path.exists(addressPrefix):
    addressPrefix='C:/GDrive/Documents/Educational/TAMU/Research/CGM Dataset/Hoover/'
pd.options.mode.chained_assignment = None  # default='warn'
plt.style.use({'figure.facecolor':'white'})


import multiprocessing
from joblib import Parallel, delayed

num_cores = multiprocessing.cpu_count()



# import warnings
# warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)

# def dfCompactor(df):
#     df['Date']=df['Date'].astype(int)
#     df['Time']=df['Time']*1000
#     df['Time']=df['Time'].astype(int)
#     df.rename(columns={"Time": "Time[ms]"})
#
#     df['GyroX']=df['GyroX'].astype(float)
#     df['GyroX']=df['GyroX']*1000*1000
#     df['GyroX']=df['GyroX'].astype(int)
#     df.rename(columns={"GyroX": "GyroX[microD/s]"})
#
#     df['GyroY']=df['GyroY'].astype(float)
#     df['GyroY']=df['GyroY']*1000*1000
#     df['GyroY']=df['GyroY'].astype(int)
#     df.rename(columns={"GyroY": "GyroY[microD/s]"})
#
#     df['GyroZ']=df['GyroZ'].astype(float)
#     df['GyroZ']=df['GyroZ']*1000*1000
#     df['GyroZ']=df['GyroZ'].astype(int)
#     df.rename(columns={"GyroZ": "GyroZ[microD/s]"})
#
#     df['AccelX']=df['AccelX'].astype(float)
#     df['AccelX']=df['AccelX']*1000*1000
#     df['AccelX']=df['AccelX'].astype(int)
#     df.rename(columns={"AccelX": "AccelX[microm/s2]"})
#
#     df['AccelY']=df['AccelY'].astype(float)
#     df['AccelY']=df['AccelY']*1000*1000
#     df['AccelY']=df['AccelY'].astype(int)
#     df.rename(columns={"AccelY": "AccelY[microm/s2]"})
#
#     df['AccelZ']=df['AccelZ'].astype(float)
#     df['AccelZ']=df['AccelZ']*1000*1000
#     df['AccelZ']=df['AccelZ'].astype(int)
#     df.rename(columns={"AccelZ": "AccelZ[microm/s2]"})
#
#     return df
#
# def dfOrganizer(df):
#     df.columns.values[2]='TimeStamp'
#
#     df.columns.values[8]='GyroX'
#     df.columns.values[9]='GyroY'
#     df.columns.values[10]='GyroZ'
#
#     df.columns.values[11]='AccelX'
#     df.columns.values[12]='AccelY'
#     df.columns.values[13]='AccelZ'
#
#     df = df.filter(['Name','TimeStamp','GyroX','GyroY','GyroZ','AccelX','AccelY','AccelZ'])
#     df['TimeStamp'] = df['TimeStamp'].astype(float)
#     df['TimeStamp']=df['TimeStamp']-1000*3600*4 #fixing the timezone
#
#     df.insert(2,'Date',float('nan'))
#     df.insert(3,'Time',float('nan'))
#
#     df['Date']=pd.to_datetime(df['TimeStamp'],unit='ms')
#     df['Time']=pd.to_datetime(df['TimeStamp'],unit='ms')
#     df['Date']=df['Date'].dt.dayofyear
#     df['Time']=df['Time'].dt.hour*3600+df['Time'].dt.minute*60+df['Time'].dt.second+df['Time'].dt.microsecond*0.001*0.001
#
#     df.drop(columns=['TimeStamp'],inplace=True)
#     return df
#
# def csvReader(addressPrefix):
#     dataFiles=[]
#     for root, dirs, files in os.walk(addressPrefix, topdown=False):
#        for name in files:
#            if '.csv' in name:
#                dataFiles.append([os.path.join(root,name)])
#     for counter,element in enumerate(dataFiles):
#         print(element)
#         rows = []
#         with open(element[0], 'r') as csvfile:
#             csvreader = csv.reader(csvfile,delimiter = "\t")
#             next(csvreader) #skipping the first junk line
#             headers = next(csvreader) #column titles
#             while '' in headers:
#                 headers.remove("")
#             if len(headers)!=17:
#                 continue
#             next(csvreader) #skipping the units
#             for row in csvreader:
#                 rows.append(row)
#         df = pd.DataFrame(rows,columns=headers)
#         participantName=element[0]
#         participantName=participantName[participantName.find('CSV1')+5:participantName.find('CSV1')+10]
#         df.insert(0,'Name',participantName)
#
#         df=dfOrganizer(df)
#         df=dfCompactor(df)
#
#         if counter==0:
#             dfTotal=df
#         else:
#             frames=[dfTotal,df]
#             dfTotal=pd.concat(frames)
#
#         # if counter==1:
#         #     break
#     dfTotal.sort_values(by=['Name', 'Date','Time'],inplace=True)
#     return dfTotal
#
# def preProcessor(dfTotal,R):
#     columns=dfTotal.columns.values
#     names=dfTotal['Name'].tolist()
#     names=list(set(names))
#
#     dfProc=pd.DataFrame([],columns=columns)
#     for counter,name in enumerate(names):
#         print(name, (counter+1)/len(names))
#         df=dfTotal[dfTotal['Name']==name]
#         for column in columns:
#             if column!='Time' and column!='Date' and column!='Name':
#                 df[column]=list(gaussian_filter1d(df[column].tolist(),sigma=R))
#             df['GyroX']=df['GyroX'].astype(float)
#
#         df['GyroX']=df['GyroX'].astype(int)
#         df['GyroY']=df['GyroY'].astype(int)
#         df['GyroZ']=df['GyroZ'].astype(int)
#         df['AccelX']=df['AccelX'].astype(int)
#         df['AccelY']=df['AccelY'].astype(int)
#         df['AccelZ']=df['AccelZ'].astype(int)
#
#         frames=[dfProc,df]
#         dfProc=pd.concat(frames)
#         nameIndex = dfTotal[(dfTotal.Name == name)].index
#         dfTotal.drop(nameIndex,inplace=True)
#     return dfProc
#
# def funcCaller(addressPrefix):
#     if os.path.exists(os.path.join(addressPrefix,'RawData.csv')):
#         dfRaw=pd.read_csv(os.path.join(addressPrefix,'RawData.csv'))
#     else:
#         dfRaw=csvReader(os.path.join(addressPrefix,'CSV1'))
#         dfRaw.to_csv(os.path.join(addressPrefix,'RawData.csv'),index=False)
#     names=dfRaw['Name'].tolist()
#     names=list(set(names))
#     print('Total particpant number=',len(names))
#
#     if os.path.exists(os.path.join(addressPrefix,'FilteredData.csv')):
#         dfProcessed=pd.read_csv(os.path.join(addressPrefix,'FilteredData.csv'))
#     else:
#         dfProcessed=preProcessor(dfRaw,R=3)
#         dfProcessed.to_csv(os.path.join(addressPrefix,'FilteredData.csv'),index=False)
#     return dfProcessed
#
# dfTotal=funcCaller(addressPrefix)

In [2]:
def pdFromatter(df):
    for counter in range(len(df)):
        tempStr=df.iloc[counter,1]
        tempVal=int(tempStr[0:2])*3600+int(tempStr[3:5])*60+int(tempStr[6:8])
        tempVal*=1000
        df.iloc[counter,1]=tempVal

        tempStr=df.iloc[counter,2]
        tempVal=int(tempStr[0:2])*3600+int(tempStr[3:5])*60+int(tempStr[6:8])
        tempVal*=1000
        df.iloc[counter,2]=tempVal
        if df.iloc[counter,1]>df.iloc[counter,2]:
            df.iloc[counter,2]+=24*3600*1000
    df.sort_values(by=['Name','Start','End'],inplace=True)

    return df

def labelReader(addressPrefix):
    labelFiles=[]
    for root, dirs, files in os.walk(addressPrefix, topdown=False):
       for name in files:
           if '.txt' in name:
               labelFiles.append([os.path.join(root,name),name])
    mealTime=[]
    sensorTiming=[]
    for element in labelFiles:
        nameTemp=element[1]
        nameTemp=nameTemp[:nameTemp.find('-events')]
        with open(element[0], 'r+') as txtfile:
            fileData = txtfile.read()
            fileData=fileData.splitlines()
            while '' in fileData:
                fileData.remove('')
            tempStart=fileData[0]
            tempStart=tempStart.split()
            tempStart=tempStart[2]

            tempEnd=fileData[-1]
            tempEnd=tempEnd.split()
            tempEnd=tempEnd[2]

            sensorTiming.append([nameTemp,tempStart,tempEnd])
            for counter in range(1,len(fileData)-1):
                tempStr=fileData[counter]
                tempStr=tempStr.split()
                mealTime.append([nameTemp,tempStr[1],tempStr[2]])

    dfMeal=pd.DataFrame(mealTime,columns=['Name','Start','End'])
    dfMeal=pdFromatter(dfMeal)

    dfTime=pd.DataFrame(sensorTiming,columns=['Name','Start','End'])
    dfTime=pdFromatter(dfTime)

    return dfMeal,dfTime

def shimmerReader(element):
    nameTemp=element[1]
    dataList=[]
    tempList=[]
    with open(element[0], mode='rb') as txtfile:
        fileData = txtfile.read()
        for i in range(int(len(fileData)/4)):
            if i%6==0 and i!=0:
                tempList.append(nameTemp)
                dataList.append(tempList)
                tempList=[]
            tempVal=fileData[i*4:(i+1)*4]
            tempVal=struct.unpack('f',tempVal)
            tempVal=tempVal[0]
            tempList.append(tempVal)
    txtfile.close()

    dfSensor=pd.DataFrame(dataList,columns=[ 'X','Y','Z','Yaw','Pitch','Roll','Name'])
    dfSensor=dfSensor[['Name','X','Y','Z','Yaw','Pitch','Roll']]
    del dataList
    return dfSensor

def timeFinder(dfSensor,dfTime):
    dfSensor.insert(1,'Time',float('nan'))
    name=dfSensor['Name'].tolist()
    name=name[0]
    dfTemp=dfTime[dfTime['Name']==name]

    if len(dfTemp)>1:
        print('More than one event file for:',name)
        return
    elif len(dfTemp)==0:
        print('No event file for:',name)
        return
    startTemp=dfTemp['Start'].tolist()
    endTemp=dfTemp['End'].tolist()
    tempTimeStamp=np.linspace(startTemp,endTemp,num=len(dfSensor))
    dfSensor['Time']=tempTimeStamp
    return dfSensor

def featureExtractor(df):
    windowLength=30*1000
    featureData=[]
    name=df['Name'].tolist()
    name=name[0]
    dfName=df[df['Name']==name]
    startTime=dfName['Time'].min()
    endTime=startTime+windowLength

    while startTime<24*3600*1000:
        dfTemp=dfName[dfName['Time']>=startTime]
        dfTemp=dfTemp[dfTemp['Time']<endTime]
        if len(dfTemp)>5*15:
            f1=abs(dfTemp['Yaw'].values)+abs(dfTemp['Roll'].values)+abs(dfTemp['Pitch'].values)
            f2=abs(dfTemp['X'].values)+abs(dfTemp['Y'].values)+abs(dfTemp['Z'].values)
            f2+=0.0001
            f1=f1/f2
            f1=np.mean(f1)
            f2=np.mean(f2)
            if np.isnan(f1):
                print('Nan F1 Value')
                continue
            featureData.append([name,startTime,endTime,f1,f2])
        startTime+=windowLength
        endTime+=windowLength
    return featureData

def labelExtractor(dfMeal,features):
    dataTotal=[]
    for feature in features:
        windowName=feature[0]
        windowStart=feature[1]
        windowEnd=feature[2]
        f1=feature[3]
        f2=feature[4]
        dfTemp=dfMeal[dfMeal['Name']==windowName]
        if len(dfTemp)==0:
            print('skipped',windowName)
            break
        eatingFlag=False
        for counter in range(0,len(dfTemp)):
            if dfTemp.iloc[counter,1]<windowEnd and dfTemp.iloc[counter,2]>windowStart:
                eatingFlag=True
                break
        dataTotal.append([windowName,f1,f2,eatingFlag])
    return dataTotal

def callerFunc(element,dfMeal,dfTime):
    print(element)
    allData=[]
    dfSensor=shimmerReader(element)
    dfSensor=timeFinder(dfSensor,dfTime)
    featureData=featureExtractor(dfSensor)
    allData.extend(labelExtractor(dfMeal,featureData))

    return allData

def main(addressPrefix):
    shimmerFiles=[]
    dfMeal,dfTime=labelReader(os.path.join(addressPrefix,'EVENTfiles'))

    for root, dirs, files in os.walk(os.path.join(addressPrefix,'SHMfiles'), topdown=False):
       for name in files:
           if '.shm' in name:
               shimmerFiles.append([os.path.join(root,name),name[:-4]])

    allData=Parallel(n_jobs=num_cores)(delayed(callerFunc)(i, dfMeal,dfTime) for i in shimmerFiles)
    allData = [ item for elem in allData for item in elem]
    dfAllData=pd.DataFrame(allData,columns=['Name','F1','F2','EatingFlag'])
    dfAllData.to_csv(r'C:\GitHub\AllDataParallel.csv',index=False)
    return allData

allData=main(addressPrefix)

In [9]:
def XGClassifier(dataList, labelList,randomSeed):
    trainData, testData, trainLabels, testLabels = train_test_split(dataList, labelList, test_size=0.25,random_state=randomSeed)
    trainData, valData, trainLabels, valLabels = train_test_split(trainData, trainLabels, test_size=0.33,random_state=randomSeed)
    recallBest=0
    for maxDepth in np.arange(2,6):
        for estimator in [20,50,80,120,180,250]:
            clf = xgb.XGBClassifier(n_estimators=estimator,max_depth=maxDepth,objective = "binary:logistic",
                                    eval_metric = "logloss",use_label_encoder =False,scale_pos_weight=5)
            clf.fit(trainData, trainLabels)
            for threshold in [0.4,0.6,0.8]:
                slidingWindowPrediction = clf.predict_proba(valData)
                slidingWindowPrediction=slidingWindowPrediction[:,1]
                slidingWindowPrediction[slidingWindowPrediction>=threshold]=1
                slidingWindowPrediction[slidingWindowPrediction<threshold]=0

                confMatrix=sklearn.metrics.confusion_matrix(valLabels,slidingWindowPrediction,normalize='all')
                accuracy=sklearn.metrics.accuracy_score(valLabels,slidingWindowPrediction)
                recall=sklearn.metrics.recall_score(valLabels,slidingWindowPrediction)
                precision=sklearn.metrics.precision_score(valLabels,slidingWindowPrediction)
                f1=sklearn.metrics.f1_score(valLabels,slidingWindowPrediction,average='weighted')

                if recall>recallBest:
                    f1Best=f1
                    maxDepthBest=maxDepth
                    estimatorBest=estimator
                    confMatrixBest=confMatrix
                    accuracyBest=accuracy
                    modelBest=clf
                    recallBest=recall
                    precisionBest=precision
                    thresholdBest=threshold
    print('Testing on validation dataset:')
    print(confMatrixBest)
    print('Accuracy',np.round(100*accuracyBest,0),'Recall',np.round(100*recallBest,0),'Precision',np.round(100*precisionBest,0))
    print('MaxDepth:',maxDepthBest,"EstimatorBest:",estimatorBest,'Best threshold:',thresholdBest)

    slidingWindowPrediction = modelBest.predict_proba(testData)
    slidingWindowPrediction=slidingWindowPrediction[:,1]
    slidingWindowPrediction[slidingWindowPrediction>=thresholdBest]=1
    slidingWindowPrediction[slidingWindowPrediction<thresholdBest]=0

    confMatrix=sklearn.metrics.confusion_matrix(testLabels,slidingWindowPrediction,normalize='all')
    accuracy=sklearn.metrics.accuracy_score(testLabels,slidingWindowPrediction)
    recall=sklearn.metrics.recall_score(testLabels,slidingWindowPrediction)
    precision=sklearn.metrics.precision_score(testLabels,slidingWindowPrediction)

    print('Testing on test dataset:')
    print(confMatrix)
    print('Accuracy',np.round(100*accuracy,0),'Recall',np.round(100*recall,0),'Precision',np.round(100*precision,0))

data=pd.read_csv(r'C:\Users\sorush.omidvar\Google Drive\Documents\Educational\TAMU\Research\CGM Dataset\Hoover\AllDataParallel.csv')
f1Data=data['F1'].values
f1Data=np.asarray(f1Data)

f2Data=data['F2'].values
f2Data=np.asarray(f2Data)

allData=np.zeros((len(f1Data),2))
allData[:,0]=f1Data
allData[:,1]=f2Data

allData[:,0]-=np.mean(allData[:,0])
allData[:,0]/=np.std(allData[:,0])

allData[:,1]-=np.mean(allData[:,1])
allData[:,1]/=np.std(allData[:,1])

allLabel=data['EatingFlag'].values
allLabel=np.asarray(allLabel,dtype=int)

XGClassifier(allData, allLabel,randomSeed=53)

c:\users\sorush.omidvar\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\sorush.omidvar\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\sorush.omidvar\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\u

Testing on validation dataset:
[[0.84878391 0.09334579]
 [0.03597185 0.02189845]]
Accuracy 87.0 Recall 38.0 Precision 19.0
MaxDepth: 2 EstimatorBest: 50 Best threshold: 0.4
Testing on test dataset:
[[0.84624223 0.093778  ]
 [0.03694175 0.02303801]]
Accuracy 87.0 Recall 38.0 Precision 20.0
